### Import Libraries
#### Please use pip install pyamaze before running this code if not already executed

In [1]:
from pyamaze import maze, agent, COLOR, textLabel
import tracemalloc as memory_trace
import time
from IPython.display import display
import pandas as pd
import copy

### Impelement MDP Policy Iteration Algortihm Functionality

In [2]:
class MDP_Policy_Iteration_Search : 
    
    def __init__(self, maze_size) : 
        self.maze_size = maze_size
    
    def load_maze(self) : 
        m = maze()
        maze_name = 'Maze_' + str(self.maze_size) + 'X' + str(self.maze_size)
        m.CreateMaze(loadMaze = maze_name + '.csv')
        return m
    
    def start_memory_tracing(self) :
        memory_trace.stop()
        memory_trace.start()
        
    def stop_memory_tracing(self) : 
        memory_size, memory_peak = memory_trace.get_traced_memory()
        return memory_size, memory_peak
    
    def initialize_maze(self) : 
        self.maze = self.load_maze()
        self.goal_node = self.maze._goal
        self.start_node = (self.maze_size, self.maze_size)
    
    def initialise_cost(self) : 
        self.transition_value = {node: 1 if node == self.maze._goal else 0 for node in self.maze.grid}
        self.transition_reward = {node: 1 if node == self.maze._goal else 0 for node in self.maze.grid}
        
        self.transition_dictionary = copy.deepcopy(self.maze.maze_map)
        for key in self.transition_dictionary : 
            for subkey in self.transition_dictionary[key] : 
                self.transition_dictionary[key][subkey] = 0
        
        self.transition_policy = {node: None if node == self.maze._goal else 'N' for node in self.maze.grid}
        self.initial_transition_value = {}
        self.initial_transition_value['N'] = 1
        self.initial_transition_value['S'] = 1
        self.initial_transition_value['E'] = 1
        self.initial_transition_value['W'] = 1
        
        self.gamma = 0.9
        self.threshold = 0.000001
        
    def execute_value_iteration(self, current_node) : 
        temporary_transition_value = {}
        temporary_transition_value['N'] = 0
        temporary_transition_value['S'] = 0
        temporary_transition_value['E'] = 0
        temporary_transition_value['W'] = 0
        temporary_node_transtion_value = {current_node : temporary_transition_value}
       
        for __direction__ in ['N', 'S', 'E', 'W']:
                    
            if self.maze.maze_map[current_node][__direction__] == 1 :

                try:
                    if __direction__ == 'N' : 
                        next_node = (current_node[0] - 1, current_node[1])

                    elif __direction__ == 'S' : 
                        next_node = (current_node[0] + 1, current_node[1])

                    elif __direction__ == 'E' : 
                        next_node = (current_node[0], current_node[1] + 1)

                    elif __direction__ == 'W' : 
                        next_node = (current_node[0], current_node[1] - 1)
                except:
                    next_node = None
                    
                if next_node is not None : 
                    next_node_value = self.transition_value[next_node]
                else :
                    next_node_value = 0
                    
                temporary_node_transtion_value[current_node][__direction__] =  self.initial_transition_value[__direction__] * (self.transition_reward[current_node] + next_node_value * self.gamma)
                                                                               
        return temporary_node_transtion_value
        
    
    def execute_mdp_policy_iteration_search(self):

        self.initialize_maze()
        self.initialise_cost()

        start_time = time.time() * 1000
        self.start_memory_tracing()
        has_value_converged = False
        has_policy_converged = False
        
        while not has_policy_converged : 
            has_policy_converged = True
            
            has_value_converged = False
            while not has_value_converged : 
                has_value_converged = True
                
                for current_node in self.maze.grid : 
                    
                    if current_node == self.goal_node : 
                        continue
                    
                    current_policy = self.transition_policy[current_node]
                    if self.maze.maze_map[current_node][current_policy] == 1 : 
                        try:
                            if current_policy == 'N' : 
                                next_node = (current_node[0] - 1, current_node[1])

                            elif current_policy == 'S' : 
                                next_node = (current_node[0] + 1, current_node[1])

                            elif current_policy == 'E' : 
                                next_node = (current_node[0], current_node[1] + 1)

                            elif current_policy == 'W' : 
                                next_node = (current_node[0], current_node[1] - 1)
                        except:
                            next_node = None
                        
                        if next_node is not None : 
                            next_node_value = self.transition_value[next_node]
                        else :
                            next_node_value = 0
                            
                        self.transition_dictionary[current_node][current_policy] =  self.initial_transition_value[current_policy] * (self.transition_reward[current_node] + next_node_value * self.gamma)
                        
                        if abs(self.transition_value[current_node] - (self.initial_transition_value[current_policy] * (self.transition_reward[current_node] + next_node_value * self.gamma))) > self.threshold : 
                            self.transition_value[current_node] = self.initial_transition_value[current_policy] * (self.transition_reward[current_node] + next_node_value * self.gamma)
                            has_value_converged = False
                        
            for current_node in self.maze.grid : 
                
                if current_node == self.goal_node : 
                    continue
                    
                current_node_transition_value = self.execute_value_iteration(current_node)
                
                current_node_transition_policy = self.find_best_transition_direction(current_node_transition_value[current_node])
                
                if self.transition_policy[current_node] != current_node_transition_policy : 
                    self.transition_policy[current_node] = current_node_transition_policy
                    has_policy_converged = False
            
        end_time = time.time() * 1000
        time_taken = (end_time - start_time)
        
        memory_size, memory_peak = self.stop_memory_tracing()
        memory_consumed = round((memory_peak/(1024*1024)), 3)
        goal_nodes = self.find_goal_nodes(self.transition_policy, self.start_node, self.goal_node)
        
        statistics_df = pd.DataFrame(columns=['Maze Size', 'Time Taken (in ms)', 'Memory Consumed (in MB)', 'Number of Cell in Shortest Path to Goal'])
        statistics_dict = {}
        statistics_dict['Maze Size'] = str(self.maze_size) + 'X' + str(self.maze_size)
        statistics_dict['Time Taken (in ms)'] = time_taken
        statistics_dict['Memory Consumed (in MB)'] = memory_consumed
        statistics_dict['Number of Cell in Shortest Path to Goal'] = len(goal_nodes) + 1
        
        self.display_mdp_value_iteration_path(goal_nodes, time_taken, memory_consumed, len(goal_nodes) + 1)
        
        statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
        
        return statistics_df 
    
    def find_goal_nodes(self, transition_policy, start_node, goal_node) : 
        goal_nodes = {}
        next_node_to_goal = [start_node]
        
        while len(next_node_to_goal) > 0 : 
            current_node = next_node_to_goal.pop()
            
            if current_node == goal_node : 
                break
            
            best_transition_policy = transition_policy[current_node]
            print(f'\nCurrent Cell: {current_node}, Best Transition Direction: {best_transition_policy}')
            if best_transition_policy == 'N' : 
                next_node = (current_node[0] - 1, current_node[1])

            elif best_transition_policy == 'S' : 
                next_node = (current_node[0] + 1, current_node[1])

            elif best_transition_policy == 'E' : 
                next_node = (current_node[0], current_node[1] + 1)

            elif best_transition_policy == 'W' : 
                next_node = (current_node[0], current_node[1] - 1)
            
            goal_nodes[current_node] = next_node
            next_node_to_goal.append(next_node)
        
        return goal_nodes
    
    def find_best_transition_direction(self, current_node) : 
        transition_values = list(current_node.values())
        directions = list(current_node.keys())
        return directions[transition_values.index(max(transition_values))]
    
    def display_mdp_value_iteration_path(self, goal_nodes, time_taken, memory_consumed, len_goal_nodes) : 
        
        goal_path = agent(self.maze, x = self.maze_size, y = self.maze_size, footprints = True, color=COLOR.cyan) 
        self.maze.tracePath({goal_path : goal_nodes}, delay = 100)
        
        textLabel(self.maze, 'Maze Size ', str(self.maze_size) + 'X' + str(self.maze_size))
        textLabel(self.maze, 'Time Taken (in ms) ', time_taken)
        textLabel(self.maze, 'Memory Consumed (in MB) ', memory_consumed)
        textLabel(self.maze, 'Number of Cell in Shortest Path to Goal ', len_goal_nodes)
        
        self.maze.run()

### Executing MDP Policy Iteration for Maze Size 20 X 20

In [3]:
mdp_policy_iteration_20 = MDP_Policy_Iteration_Search(20)

statistics = mdp_policy_iteration_20.execute_mdp_policy_iteration_search()

statistics = statistics.style.applymap(lambda x:'white-space:nowrap')
display(statistics)



Current Cell: (20, 20), Best Transition Direction: N

Current Cell: (19, 20), Best Transition Direction: W

Current Cell: (19, 19), Best Transition Direction: W

Current Cell: (19, 18), Best Transition Direction: N

Current Cell: (18, 18), Best Transition Direction: N

Current Cell: (17, 18), Best Transition Direction: E

Current Cell: (17, 19), Best Transition Direction: N

Current Cell: (16, 19), Best Transition Direction: E

Current Cell: (16, 20), Best Transition Direction: N

Current Cell: (15, 20), Best Transition Direction: N

Current Cell: (14, 20), Best Transition Direction: N

Current Cell: (13, 20), Best Transition Direction: N

Current Cell: (12, 20), Best Transition Direction: N

Current Cell: (11, 20), Best Transition Direction: W

Current Cell: (11, 19), Best Transition Direction: W

Current Cell: (11, 18), Best Transition Direction: W

Current Cell: (11, 17), Best Transition Direction: S

Current Cell: (12, 17), Best Transition Direction: W

Current Cell: (12, 16), Bes

,Maze Size,Time Taken (in ms),Memory Consumed (in MB),Number of Cell in Shortest Path to Goal
0,20X20,194.478516,0.028000,65


### Executing MDP Policy Iteration for Maze Size 30 X 30

In [5]:
mdp_policy_iteration_30 = MDP_Policy_Iteration_Search(30)

statistics = mdp_policy_iteration_30.execute_mdp_policy_iteration_search()

statistics = statistics.style.applymap(lambda x:'white-space:nowrap')
display(statistics)



Current Cell: (30, 30), Best Transition Direction: N

Current Cell: (29, 30), Best Transition Direction: W

Current Cell: (29, 29), Best Transition Direction: W

Current Cell: (29, 28), Best Transition Direction: N

Current Cell: (28, 28), Best Transition Direction: E

Current Cell: (28, 29), Best Transition Direction: E

Current Cell: (28, 30), Best Transition Direction: N

Current Cell: (27, 30), Best Transition Direction: N

Current Cell: (26, 30), Best Transition Direction: W

Current Cell: (26, 29), Best Transition Direction: N

Current Cell: (25, 29), Best Transition Direction: E

Current Cell: (25, 30), Best Transition Direction: N

Current Cell: (24, 30), Best Transition Direction: W

Current Cell: (24, 29), Best Transition Direction: W

Current Cell: (24, 28), Best Transition Direction: N

Current Cell: (23, 28), Best Transition Direction: E

Current Cell: (23, 29), Best Transition Direction: E

Current Cell: (23, 30), Best Transition Direction: N

Current Cell: (22, 30), Bes

,Maze Size,Time Taken (in ms),Memory Consumed (in MB),Number of Cell in Shortest Path to Goal
0,30X30,565.486572,0.046000,99


### Executing MDP Policy Iteration for Maze Size 40 X 40

In [6]:
mdp_policy_iteration_40 = MDP_Policy_Iteration_Search(40)

statistics = mdp_policy_iteration_40.execute_mdp_policy_iteration_search()

statistics = statistics.style.applymap(lambda x:'white-space:nowrap')
display(statistics)



Current Cell: (40, 40), Best Transition Direction: W

Current Cell: (40, 39), Best Transition Direction: N

Current Cell: (39, 39), Best Transition Direction: W

Current Cell: (39, 38), Best Transition Direction: W

Current Cell: (39, 37), Best Transition Direction: S

Current Cell: (40, 37), Best Transition Direction: W

Current Cell: (40, 36), Best Transition Direction: W

Current Cell: (40, 35), Best Transition Direction: W

Current Cell: (40, 34), Best Transition Direction: N

Current Cell: (39, 34), Best Transition Direction: W

Current Cell: (39, 33), Best Transition Direction: W

Current Cell: (39, 32), Best Transition Direction: N

Current Cell: (38, 32), Best Transition Direction: N

Current Cell: (37, 32), Best Transition Direction: W

Current Cell: (37, 31), Best Transition Direction: W

Current Cell: (37, 30), Best Transition Direction: W

Current Cell: (37, 29), Best Transition Direction: N

Current Cell: (36, 29), Best Transition Direction: N

Current Cell: (35, 29), Bes

,Maze Size,Time Taken (in ms),Memory Consumed (in MB),Number of Cell in Shortest Path to Goal
0,40X40,1389.273926,0.085000,127
